In [1]:
import tensorflow as tf

2022-12-20 21:43:00.850160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 21:43:01.456686: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-20 21:43:01.539396: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 21:43:01.539429: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
import os
import numpy as np
import random

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
PREPROCESSING_ARGS = {
    'downsampling_rate': 16000,
    'frame_lenght_in_s': 0.032,
    'frame_step_in_s':0.032,
    'num_mel_bins': 25,
    'lower_f': 0,
    'upper_f': 8000,
    'num_coefficients': 13
}

TRAINING_ARGS = {
    'batch_size': 8,
    'epochs': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
}



alpha = 0.7
initial_sparsity = 0.5
final_sparsity = 0.85

In [4]:
go_stop_train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
go_stop_val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
go_stop_test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])

2022-12-20 21:43:09.449423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-20 21:43:09.449467: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-20 21:43:09.449486: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-f6dc309a-5f8d-46e7-b3a6-26eb71e83e3d): /proc/driver/nvidia/version does not exist
2022-12-20 21:43:09.449795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
num_train_files = len(go_stop_train_ds)
num_val_files = len(go_stop_val_ds)
num_test_files = len(go_stop_test_ds)

print('Training set size:', num_train_files)
print('Validation set size:', num_val_files)
print('Test set size:', num_test_files)

Training set size: 1600
Validation set size: 200
Test set size: 200


In [6]:
from preprocessing import get_mfccs
from functools import partial

LABELS = ['go', 'stop']

get_frozen_mfccs = partial(get_mfccs, **PREPROCESSING_ARGS)

for x,y in go_stop_train_ds.map(get_frozen_mfccs).take(1):
    SHAPE = x.shape

def preprocess(filename):
    signal, label = get_frozen_mfccs(filename)
    signal.set_shape(SHAPE)
    signal = tf.expand_dims(signal, -1)
    signal = tf.image.resize(signal, [32, 32])
    label_id = tf.argmax(label == LABELS)
    #print(type(signal))
    signal = tf.cast(signal, tf.float16)
    label_id = tf.cast(label_id, tf.int8)

    return signal, label_id

2022-12-20 21:43:11.661388: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-12-20 21:43:11.661627: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 AVX512F FMA
2022-12-20 21:43:13.941489: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:13.945094: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:13.945426: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library


In [7]:
batch_size = TRAINING_ARGS['batch_size']
epoch = TRAINING_ARGS['epochs']



train_ds = go_stop_train_ds.map(preprocess).batch(batch_size).cache()
val_ds = go_stop_val_ds.map(preprocess).batch(batch_size).cache()
test_ds = go_stop_test_ds.map(preprocess).batch(batch_size).cache()

2022-12-20 21:43:17.745795: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:17.748312: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:17.748550: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:19.661131: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:19.742207: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:19.742427: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-20 21:43:21.462523: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed 

In [8]:
for example_batch, example_labels in train_ds.take(1):
    print(example_batch.shape)
    print(example_labels)

(8, 32, 32, 1)
tf.Tensor([0 1 0 1 0 0 0 0], shape=(8,), dtype=int8)
2022-12-20 21:43:24.143358: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[2, 1],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(64 * alpha), kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(64 * alpha), kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=2),
    tf.keras.layers.Softmax()
])

In [10]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epoch * 0.2)
end_step = int(len(train_ds) * epoch)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=initial_sparsity,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step,
    )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

In [11]:
model_for_pruning.summary(), tf.keras.utils.plot_model(model_for_pruning, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 15, 30, 89)       1604      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 15, 30, 89)       357       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 15, 30, 89)       1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_depthwi  (None, 15, 30, 89)       802       
 se_conv2d (PruneLowMagnitud                                     
 e)                                                     

In [12]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epoch,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)
#model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epoch, validation_data=val_ds, callbacks=callbacks)
#history = model.fit(train_ds, epochs=epoch, validation_data=val_ds)

Epoch 1/20
200/200 [==============================] - 47s 154ms/step - loss: 0.6245 - sparse_categorical_accuracy: 0.6637 - val_loss: 0.6233 - val_sparse_categorical_accuracy: 0.7400
Epoch 2/20
200/200 [==============================] - 23s 115ms/step - loss: 0.4366 - sparse_categorical_accuracy: 0.7987 - val_loss: 0.2514 - val_sparse_categorical_accuracy: 0.9050
Epoch 3/20
200/200 [==============================] - 22s 112ms/step - loss: 0.2903 - sparse_categorical_accuracy: 0.8900 - val_loss: 0.2332 - val_sparse_categorical_accuracy: 0.9200
Epoch 4/20
200/200 [==============================] - 23s 115ms/step - loss: 0.2029 - sparse_categorical_accuracy: 0.9256 - val_loss: 0.2552 - val_sparse_categorical_accuracy: 0.8850
Epoch 5/20
200/200 [==============================] - 23s 115ms/step - loss: 0.1670 - sparse_categorical_accuracy: 0.9431 - val_loss: 0.1841 - val_sparse_categorical_accuracy: 0.9500
Epoch 6/20
200/200 [==============================] - 23s 116ms/step - loss: 0.1393 -

In [13]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)
#test_loss, test_accuracy = model.evaluate(test_ds)

25/25 [==============================] - 1s 26ms/step - loss: 0.0569 - sparse_categorical_accuracy: 0.9750


In [14]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training loss: {training_loss:.3f}')
print(f'Training accuracy: {training_accuracy * 100:.2f}%')
print(f'Validation loss: {val_loss:.3f}')
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.3f}')
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Training loss: 0.032
Training accuracy: 98.94%
Validation loss: 0.102
Validation accuracy: 96.50%
Test loss: 0.057
Test accuracy: 97.50%


In [15]:
from time import time
timestamp = time()
MODEL_NAME = int(timestamp)

In [16]:
saved_model_dir = f'saved_models/{MODEL_NAME}'

if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)

model.save(saved_model_dir)

INFO:tensorflow:Assets written to: saved_models/1671573034/assets
INFO:tensorflow:Assets written to: saved_models/1671573034/assets


In [17]:
#!ls saved_models/

In [18]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

2022-12-20 21:50:36.254068: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-20 21:50:36.254103: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-20 21:50:36.254633: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/1671573034
2022-12-20 21:50:36.257430: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-20 21:50:36.257473: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/1671573034
2022-12-20 21:50:36.263328: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-20 21:50:36.264661: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-20 21:50:36.297985: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./save

In [19]:
tflite_models_dir = './tflite_models'

if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name

'./tflite_models/1671573034.tflite'

In [20]:
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

In [21]:
downsampling_rate = PREPROCESSING_ARGS['downsampling_rate']
sampling_rate_int64 = tf.cast(downsampling_rate, tf.int64)
frame_length = int(downsampling_rate * PREPROCESSING_ARGS['frame_lenght_in_s'])
frame_step = int(downsampling_rate * PREPROCESSING_ARGS['frame_step_in_s'])
num_spectrogram_bins = frame_length // 2 + 1
num_mel_bins = PREPROCESSING_ARGS['num_mel_bins']
lower_frequency = PREPROCESSING_ARGS['lower_f']
upper_frequency = PREPROCESSING_ARGS['upper_f']
num_coefficients = PREPROCESSING_ARGS['num_coefficients']

linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    num_mel_bins=num_mel_bins,
    num_spectrogram_bins=num_spectrogram_bins,
    sample_rate=downsampling_rate,
    lower_edge_hertz=lower_frequency,
    upper_edge_hertz=upper_frequency
)

In [22]:
interpreter = tf.lite.Interpreter(model_path=f'./tflite_models/{MODEL_NAME}.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Number of inputs:', len(input_details))
print('Input name:', input_details[0]['name'])
print('Input index:', input_details[0]['index'])
print('Number of output:', len(output_details))
print('Output name:', output_details[0]['name'])
print('Output index:', output_details[0]['index'])

Number of inputs: 1
Input name: serving_default_input_1:0
Input index: 0
Number of output: 1
Output name: StatefulPartitionedCall:0
Output index: 21
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [23]:
from glob import glob
pattern = ('./msc-test/go*', './msc-test/stop*')
filenames = sum([glob(pat) for pat in pattern], [])
print(len(filenames))

200


In [24]:
from time import time

avg_preprocessing_latency = 0
avg_model_latency = 0
latencies = []
accuracy = 0.0

for filename in filenames:
    audio_binary = tf.io.read_file(filename)

    # NEED ONLY FOR TESTING
    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    true_label = file_parts[0]
    true_label = true_label.numpy().decode()

    # PRE-PROCESSING (LOG-MEL SPECTROGRAM)
    start_preprocess = time()
    audio, sampling_rate = tf.audio.decode_wav(audio_binary)
    audio = tf.squeeze(audio)

    zero_padding = tf.zeros(sampling_rate - tf.shape(audio), dtype=tf.float32)
    audio_padded = tf.concat([audio, zero_padding], axis=0)

    if downsampling_rate != sampling_rate:
        audio_padded = tfio.audio.resample(audio_padded, sampling_rate_int64, downsampling_rate)

    stft = tf.signal.stft(
        audio_padded,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    spectrogram = tf.abs(stft)

    mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)[...,:num_coefficients]
    
    mfccs = tf.expand_dims(mfccs, 0)  # batch axis
    mfccs = tf.expand_dims(mfccs, -1)  # channel axis
    mfccs = tf.image.resize(mfccs, [32, 32])
    end_preprocess = time()

    interpreter.set_tensor(input_details[0]['index'], mfccs)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    end_inference = time()

    top_index = np.argmax(output[0])
    predicted_label = LABELS[top_index]

    accuracy += true_label == predicted_label
    avg_preprocessing_latency += (end_preprocess - start_preprocess)
    avg_model_latency += (end_inference - end_preprocess)
    latencies.append(end_inference - start_preprocess)

In [25]:
accuracy /= len(filenames)
avg_preprocessing_latency /= len(filenames)
avg_model_latency /= len(filenames)
total_latency = np.median(latencies)

import os

model_size = os.path.getsize(f'./tflite_models/{MODEL_NAME}.tflite')

In [26]:
print(f'Accuracy: {accuracy * 100.:.3f}%')
print(f'Model size: {model_size / 1024:.1f} KB')
print(f'Preprocessing Latency: {1000 * avg_preprocessing_latency:.1f} ms')
print(f'Model Latency: {1000 * avg_model_latency:.1f} ms')
print(f'Total Latency: {1000 * total_latency:.1f} ms')

Accuracy: 97.500%
Model size: 65.8 KB
Preprocessing Latency: 5.4 ms
Model Latency: 0.9 ms
Total Latency: 6.1 ms


In [27]:
import zipfile

tflite_models_dir = './tflite_models'

if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

In [28]:
tflite_model_name

'./tflite_models/1671573034.tflite'

In [29]:
pruned_tflite_size = os.path.getsize(tflite_model_name) / 1024
pruned_zip_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024

print(f'Original TFLite Size (pruned model): {pruned_tflite_size:.2f} KB')
print(f'ZIP TFLite Size (pruned model): {pruned_zip_size:.2f} KB')

Original TFLite Size (pruned model): 65.78 KB
ZIP TFLite Size (pruned model): 23.66 KB


In [30]:
import pandas as pd
from time import time

#timestamp = time()

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

output_dict = {
    'timestamp': int(timestamp),
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'final_sparsity': final_sparsity,
    'test_accuracy': test_accuracy,
    'size_zip': pruned_zip_size,
    'tflite_size': pruned_tflite_size,
    'train_accuracy': training_accuracy,
    'latency' : f'{1000*total_latency:.1f} ms',
    'alpha': alpha

}
output_dict

df = pd.DataFrame([output_dict])

In [31]:
import os
output_path = 'MFCCS_pruned_test.csv'

df.to_csv(output_path, mode='a', header=not (os.path.exists(output_path)), index=False)

In [32]:
saved_model_dir = f'saved_models/{MODEL_NAME}'

if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)

model_for_export.save(saved_model_dir)

INFO:tensorflow:Assets written to: saved_models/1671573034/assets
INFO:tensorflow:Assets written to: saved_models/1671573034/assets


In [33]:
print(f'Total latency: {1000*total_latency:.1f} ms')
print(f'Train Accuracy: {100*training_accuracy:.2f}%')
print(f'Size Zipped: {pruned_zip_size:.2f}kb')

Total latency: 6.1 ms
Train Accuracy: 98.94%
Size Zipped: 23.66kb


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6dc309a-5f8d-46e7-b3a6-26eb71e83e3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>